### Запуск QtLab

Для начала надо добавить пути со скриптами и драйверами устройств, подключить библиотеки numpy и matplotlib.

In [3]:
%run D:\qtlab_replacement\init.py
#%matplotlib qt4
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *

ERROR:root:File `'D:\\qtlab_replacement\\init.py'` not found.


ModuleNotFoundError: No module named 'PyQt4'

Чтобы не держать константы далеко в коде, мы записываем сюда параметры кубитов.

In [2]:
#Параметры кубитов.
Qubits = {1: {'Fr': 9.1507e9,'F01': 5.875e9, 'I0': 0.,'Ispan': 0.,'P': 0,'Pex': -2,'Span': 200e6},
          2: {'Fr': 9.576e9 ,'F01': 6.74974e9, 'P':-30},
          5: {'Fr': 9.515e9,'F01': 6.18956e9, 'I0': 0.,'Ispan': 0.,'P': -30,'Pex': -2,'Span': 200e6},
          6: {'Fr': 9.515e9,'F01': 5.909e9, 'I0': 0.,'Ispan': 0.,'P': -10,'Pex': -2,'Span': 200e6}}
          #7: {'Fr': 9.5155e9,'F01': 6.095e9, 'I0': 0.,'Ispan': 0.,'P': -10,'Pex': -2,'Span': 200e6}}
#           5: {'Fr': 9.5155e9,'F01': 6.19719e9, 'I0': 0.,'Ispan': 0.,'P': -10,'Pex': -2,'Span': 200e6}}
qubit_id = 1
#5.91557

In [3]:
awg = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


# Спектроскопия

Инстантизируем драйверы устройств для векторного анализатора ('pna') и генератора сигналов ('lo1').

In [4]:
pna = Agilent_N5242A('pna', address = 'PNA')
lo1 = Agilent_E8257D('lo1', address = 'PSG1')

In [5]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

True

## Спектроскопия через смесители

Если подключена импульсная схема, а хочется сделать спектроскопию - то нет проблем! Единственное что нужно сделать, это, во-первых, открыть смесители (подав на них, скажем, 0.3В), и во-вторых, обратить внимание на мощность на выходе. Она будет ниже чем если делать спектроскопию нормально. Ну и шумов тоже получится больше - лучше всего не подавать какие-то сигналы с AWG, а задать оффсеты. Тогда вроде бы шумов меньше получается.

Что тут хорошо: можно переключаться между режимами не выходя из офиса (или из вообще не вставая на кровати). Этот режим очень популярен при измерениях в Черноголовке среди людей, которые не любят в эту самую Черноголовку ездить.
Тем не менее иногда полезно заходить в лабу и посмотреть что же на самом деле там происходит.

In [1]:
# AWG для того чтобы открыть смесители
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
Vdc_q = 0.2
Vdc_r = 0.2
# Задаём постояннmtые смещения на каналы 1 и 3
awg.set_waveform([0]*awg_tek.get_nop(), channel=1)
awg.set_waveform([0]*awg_tek.get_nop(), channel=2)
awg.set_waveform([0]*awg_tek.get_nop(), channel=3)
awg.set_waveform([0]*awg_tek.get_nop(), channel=4)
awg_tek.set_offset(Vdc_q, channel=1)
awg_tek.set_offset(Vdc_q, channel=2)
awg_tek.set_offset(Vdc_r, channel=3)
awg_tek.set_offset(Vdc_r, channel=4)

# Сами импульсы надо при этом выключить! Будет меньше шуметь (но то не точно).
awg_tek.stop()

NameError: name 'Tektronix_AWG5014' is not defined

## Резонаторная спектроскопия

In [7]:
lo1.set_status(0)
pna.set_status(1)
pna.set_power(-30) # 
pna.set_nop(501)
pna.set_xlim(8.0e9, 10.0e9)
pna.set_bandwidth(200)
#freqs = pna.get_freqpoints()
#S21 = pna.get_tracedata()

True

In [8]:
meas = sweep.sweep(pna, ([0], lambda x: None, 'None'), filename='Single Tone spectroscopy #2')

Started at:  Jan 13 2018 15:51:23

Elapsed time: 0 h 0 m 2.75 s


## Резонатор от мощности (проверка нелинейности резонатора)

In [13]:
pna.set_power(-10) # 
pna.set_nop(101) 
#pna.set_xlim(9.8e9, 9.81e9)
pna.set_xlim(9.1475e9, 9.1525e9)
pna.set_bandwidth(30)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()

In [14]:
resonator_powers = np.arange(16, -20, -1)
meas = sweep.sweep(pna, (resonator_powers, pna.set_power, 'Power, dBm'), filename='Single Tone spectroscopy power dependent #4')

Started at:  Jan 19 2018 16:58:24


KeyboardInterrupt: 

## Фитуем добротность резонатора

In [ ]:
from resonator_tools import circuit
#plt.plot(freqs, scipy.signal.detrend(np.unwrap(S21[1])))
fitter =circuit.reflection_port(f_data=freqs, z_data_raw=S21[0]*np.exp(1j*S21[1]))
fitter.autofit()
fitter.plotall()

In [ ]:
fitter.fitresults['Qc_err']

In [ ]:
plt.plot(freqs, np.abs(np.gradient(S21[0]*np.exp(1j*S21[1]))))

In [ ]:
powers = np.linspace(-65, 5, 15)
sweep.sweep(pna, (powers, pna.set_power, 'readout power'), filename='Resonator power spectroscopy')

### 1D скан при фиксированной мощности

In [ ]:
qubit_id = 5
pna.set_nop(1)
#pna.set_xlim(9.5565e9, 9.5565e9)

In [13]:
#Измерение
lo1.set_status(1)
ex_freqs = np.arange(5.76e9, 5.78e9, 1e6)

pna.set_xlim(Qubits[qubit_id]['Fr'], Qubits[qubit_id]['Fr'])
pna.set_nop(1)
pna.set_bandwidth(1)
pna.set_average_mode('POINT')
pna.set_average(0)
pna.set_averages(4)
pna.set_power (0)
lo1.set_power (14)
#current_src.set_current(-2.5e-3)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'), filename = "ex_{:.1f}dBm".format(lo1.get_power()))
except KeyboardInterrupt: pass 

Started at:  Jan 19 2018 15:16:48

Elapsed time: 0 h 0 m 23.98 s


In [ ]:
#Измерение
lo1.set_status(1)
ex_freqs = np.arange(5.8e9, 6.3e9, 2e6)

pna.set_xlim(9.45e9, 9.75e9)
pna.set_nop(1001)
pna.set_bandwidth(501)
pna.set_average_mode('POINT')
pna.set_average(1)
pna.set_averages(1)
pna.set_power (-5)
lo1.set_power (13)
#current_src.set_current(-2.5e-3)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "ex_{:.1f}dBm #4".format(lo1.get_power()))
except KeyboardInterrupt: pass 

##### 2D скан (мощность второго тона, частота второго тона)

In [ ]:
Qubit_id = 6
ex_freqs = np.arange(5.5e9,6.3e9, 5e6)
ex_powers = np.arange(-16, 1, 1)

pna.set_xlim(Qubits[Qubit_id]['Fr'], Qubits[Qubit_id]['Fr'])
pna.set_nop(1)
pna.set_power(Qubits[Qubit_id]['P'])
pna.set_bandwidth(5)
#lo1.set_power(-20)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Second tone power'), (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "#2{:.1f}dBm".format(pna.get_power()),\
           header = header)
lo1.set_status(0)

In [ ]:
np.arange(-16, 1, 2)

# Импульсы

### Создаём девайсы

Инстантизация драйверов устройств.
sa - анализатор спектра,
awg_tek - геннератор сигналов произвольной формы,
adc - аналогово-цифровой преобразователь.

In [3]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = lo1
lo_ro = pna
#lo_ro = Labbrick(name='lo_ro', serial=15257)
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')

NameError: name 'Agilent_N9030A' is not defined

### Загружаем импульсные скрипты всякие

In [8]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)

<module 'sweep' from 'D:\\qtlab_replacement\\scripts\\sweep.py'>

### Осцилляторы

Генераторы опорных волн для смесителей.
На вход надо подавать около 13 дБ (точка децибельной компрессии).
В качестве источника можно использовать agilent или labbrick. У labbrick фазовые шумы, лучше использвать agilent.
Векторный анализатор можно включить в режим постоянной волны, тогда его тоже можно использовать.

In [9]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
#lo_ex_pow = 14
lo_ex_pow = 14
#lo_ex = lo1
lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
#lo_ro_pow = 16
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

#lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

True

### AWGшки

In [10]:
# промежуточные частоты для гетеродинной схемы:
#ex_if = 125e6
ex_if = 97.076e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 0.7 В на считывание, 0.4 В на возбуждение.
for channel in range(3,5):
    awg_tek.set_amplitude(0.5, channel=channel)
    awg_tek.set_offset(0.0, channel=channel)
for channel in range(1,3):
    awg_tek.set_amplitude(0.5, channel=channel)
    awg_tek.set_offset(0.0, channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 1, 2, lo_ex)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(Qubits[qubit_id]['F01'])
iq_ro.set_frequency(Qubits[qubit_id]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-1, channel=1)
awg_tek.set_marker2_low(-1, channel=1)
awg_tek.set_marker1_high(2, channel=1)
awg_tek.set_marker2_high(2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды

ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


### Оцифровщик

In [1]:
# настройки оцифровщика
adc.reset()            
adc.set_timeout(10000)
adc.set_clock(ro_clock)

#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(100)
adc.set_input_amp_ch1(100)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())
# Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
# выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
adc_reducer.extra_opts['scatter'] = True

NameError: name 'adc' is not defined

### Измерение двухтонового спектра импульсами

In [ ]:
#iq_ro.awg_I.__set_dc(1.0)
frequencies = np.arange(5.7e9, 6.0e9, 2e6)
iq_ex.set_if(0.0)
iq_ex._set_dc(0.1)

measurement = sweep.sweep(adc_reducer, (frequencies, iq_ex.set_frequency, 'Second tone frequency'), 
                          filename='Two-tone mixer passthrough')

In [29]:
adc.stop()

### Калибровка миксеров

In [1]:
#sa.set_ref(0)
iq_ex.get_calibration(sa)
iq_ro.get_calibration(sa)

NameError: name 'iq_ex' is not defined

### Длительность и амплитуда считывающих импульсов

In [13]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(8./7.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*7/8
    print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [14]:
 
# ro_amplitude = 0.5 Qubit #2 preliminary
# ro_amplitude = 0.3 Qubit #3 preliminary
ro_dac_amplitude = 0.9
ro_dac_length = 0.3e-6 # вообще считывающие импульсы такой длины? тогда что такое adc 
# Ответ: ro_dac_length - это длительность генерируемого прямоугольного импульса
# ro_adc_length - это минимальное время, в течении которого мы считываем этот импульс, когда он прошёл через криостат.
# В криостате импульс деформируется и становится длинее. Сколько времени хотите его интегрировать, столько и ставьте.
# Наверное, имеет смысл сюда поставить на 1-2 времени затухания фотона в резонатор больше, чем ro_dac_length
#ro_adc_length = ro_dac_length*3
#ro_adc_length=0.8e-6 use it for bench in the end
ro_adc_length=0.8e-6 #
#adc.set_nums(30000)
#adc.set_software_nums_multi(1)
adc.set_nums(20000)
set_adc_nop(ro_adc_length)
adc.set_nums(50000)
#ro_adc_length
#adc.set_nums

512


True

In [21]:
m = adc.measure()
m['Voltage'].shape


(40000, 1024)

In [ ]:
adc.stop()

### Прохождение считывающих испульсов разной амплитуды

In [37]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0.0, 1.0, 21), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

Started at:  Jan 19 2018 17:22:09


KeyboardInterrupt: 

### Осцилляции Раби

In [39]:
ex_amplitude = 1.0#смотрим на график в S21-phase ?? как жить что частота от этого зависит
pause_length = 16e-9#пауза между возб.импульсом и считывающим
lengths = np.linspace(0.0e-6, 1.0e-6,51)#время наблюдения осцилляций,для фитования частоты достатоно пары периодов
readout_begin = np.max(lengths)
lo1.set_status(True)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length', 's'), filename='Rabi')
measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters_rabi['phase'], 
                                                                             fitted_parameters_rabi['freq'], 
                                                                             fitted_parameters_rabi['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

Started at:  Jan 19 2018 17:22:17


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]
Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq\backend\cython\checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc (zmq\backend\cython\message.c:6078)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
ex_amplitude = 0.2#смотрим на график в S21-phase ?? как жить что частота от этого зависит
pause_length = 16e-9#пауза между возб.импульсом и считывающим
lengths = np.linspace(0.0e-6, 0.5e-6,101)#время наблюдения осцилляций,для фитования частоты достатоно пары периодов
#lengths = [1e-8]
ro_dac_amplitude=0.9
readout_begin = np.max(lengths)
lo1.set_status(True)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
    
frequencies = np.linspace(5.890e9,5.920e9, 61)
try:
    iq_ex.ignore_calibration_drift = True
    measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length', 's'), 
                          (frequencies, iq_ex.set_uncal_frequency, 'Frequency', 'Hz'), filename='Rabi 2D')
finally:
    iq_ex.ignore_calibration_drift = False

In [ ]:
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

In [ ]:
Rabi_fr=fitted_parameters_rabi['freq']
T_pi=np.pi/Rabi_fr
T_pi_half=T_pi/2
T_pi,T_pi_half

In [ ]:
measurement

In [ ]:
measurement_fitted

In [ ]:
adc.stop()

In [ ]:
det=np.sqrt(4*Rabi_fr**2-Qubits[4]['F01']**2)
det

### Осцилляции Рамзея

In [15]:
ex_amplitude = 1.0
# pi2_pulse = 7e-9 #Q2
pi2_pulse = 25e-9 #Q4
#pi2_pulse = T_pi_half #Q4
pause_length = 16e-9
delays = np.linspace(0e-9, 1e-6, 101)#последовательность задержек между двумя пи/2 импульсами
target_freq_offset = 12e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi2_pulse),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)),#зачем фаза? 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay', 's'), filename='Ramsey')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

Started at:  Jan 19 2018 17:27:12


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]
root: ERROR    Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\qtlab_replacement\scripts\sweep.py", line 151, in sweep
    setter(val)
  File "<ipython-input-15-d23d18081d3c>", line 22, in set_delay
    pg.set_seq(sequence)
  File "D:\qtlab_replacement\scripts\tomography.py", line 57, in set_seq
    channel_device.set_waveform(pulse_shape[channel])
  File "D:\qtlab_replacement\scripts\awg_iq.py", line 117, in set_waveform
    self.__set_waveform_IQ_cmplx(waveform_I+1j*waveform_Q)
  File "D:\qtlab_replacement\scripts\awg_iq.py", line 89, in __set_waveform_IQ_cmplx
    self.awg_I.set_waveform(waveform_I, channel=self.awg_ch_I)
  File "D:\qtlab_replacement\src\instrument.py", line 199, in <lambda>
    func = lambda val, **lopts: self.set(name, val, **lopts)
  File "D:\qtlab_replacement\src\instrument.py", line 700, in set
    val = self._set_value(name, value, **kwargs)
  File "D:\qtlab_replacement\src\instrument.py", line 659, in _set_value
    ret = func(value, **kwargs)
  File "D:\qtlab_replacement\ins

TypeError: must be str, not list

In [ ]:
plt.close()

In [45]:
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

### Затухание свободной индукции (T1)

In [1]:
ex_amplitude = 0.8
pi_pulse = 22e-6
pause_length = 16e-9

In [2]:
delays = np.linspace(0e-6,5e-6, 101)#серия задержек после возбуждения перед считыванием состояния
readout_begin = np.max(delays)+pi_pulse#зачем он тут прибавляет, а потом в коде вычитает? 
# раньше смысл был в том, чтобы менять длительность возбуждающей последовательности, не сдвигая считывание. То есть
# дополнительные импульсы приделываются до считывания (котороене двигается), а не считывание приделывается к концу.
# Сейчас это ещё раз реализовано в коде, которые делает последовательности импульсов (pg.set_seq). Если вдруг у нас будет
# много считываний, то там тоже надо будет мутить какую-нибудь синхронизацию.
def set_delay(delay):
    #awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi_pulse),
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
#save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

NameError: name 'np' is not defined

In [36]:
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
fitted_parameters

[2.0716193171851168e-05, 0.03415338179351368, 0.017645688119862345, -0.029301720020808462, -0.03698338108364822, 0.03430866135393292, 0.01660519463813009, -0.034350198812750775, -0.037587379378776035]


{'amplitudes': array([-0.00022143,  0.00090122,  0.00448393,  0.00051385,  0.03326184,
         0.01821135, -0.03264307, -0.03852435]),
 'decay': 4.4857323039310028e-06}

In [ ]:
adc.stop()

In [37]:
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

# Optimizing readout pulse settings

In [61]:
#+ 
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
# Кубит #4, импульсы 8 нс
#pause_length = 2e-9
#length = 10e-9 # Длительность импульса 
#sigma = 2.5e-9 # Срез импульса (в сигмах) = Длительност/сигма
#amp=0.893
#amp_y=0.880
#anharmonicity = -200e6
#alpha_dimensionless = 0.724
# Кубит #4, импульсы 16 нс
#pause_length = 0e-9
#length = 16e-9 # Длительность импульса 
#sigma = 3e-9 # Срез импульса (в сигмах) = Длительност/сигма
#amp=0.322
#anharmonicity = -200e6
#alpha_dimensionless = 0.909#553
#alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
#xy_acorr = 0.003
#amp_y_raw=0.322#
amp=0.402
amp2=0.834
length=10e-9
sigma=2e-9
anharmonicity=-227e6
pause_length=0
# alpha_dimensionless =1.035
alpha_dimensionless = -0.8
alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
xy_acorr =0.0
amp_y_raw=0.3245
amp_y=amp_y_raw*np.exp(1j*xy_acorr)

In [19]:
#parametres
ampx_pi_half
ampx_pi
ampy_pi_half
ampy_pi
length=16e-9
sigma=3e-9
anharmonicity=-227e6
alpha_dimensionless = -1
alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)

NameError: name 'ampx_pi_half' is not defined

In [16]:
# To calibrate we need a pi-pulse. Two choices are available, either we use a rectangular pulse or a gaussian pulse.
# The pulse is used to excite the qubit with high probability.
pi_pulse = 56e-9;
ex_amplitude=.9;
ex_calib_seq = [pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude)] # rectangular pulse
#ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma), pg.p(None, pause_length)]*2 # gaussian pulse
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc.set_software_nums_multi(4)
adc_sz.repeat_samples = 1

adc.save_last_samples = True

adc.set_nums(50000)

target_min = lambda: 1-adc_sz.measure()['Calibrated fidelity binary']

class ro_pulse_generator:
    def __init__(self, adc_sz, ro_dac_amplitude, ro_dac_length):
        self.ro_dac_amplitude = ro_dac_amplitude
        self.ro_dac_length = ro_dac_length
        self.adc_sz = adc_sz
    def make_ro_seq(self):
        adc_sz.ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', self.ro_dac_length, pg.rect, self.ro_amplitude)]
    def set_ro_length(self, ro_dac_length):
        self.ro_dac_length = ro_dac_length
        self.make_ro_seq()
    def set_ro_amplitude(self, ro_amplitude):
        self.ro_amplitude = ro_amplitude
        self.make_ro_seq()

ro_dac_lengths = np.linspace(0, 0.05e-6, 6)
ro_dac_amplitudes = np.linspace(0.0,1.0, 11)
#ro_dac_length = 300e-9

ro_pulse_generator_inst = ro_pulse_generator(adc_sz, ro_dac_amplitude, ro_dac_length)
#solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 3e-8), 
#                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.9), maxfun=30)

#ro_dac_length = solution[0]
#ro_dac_amplitude = solution[1]

In [17]:
ro_amplitude_calib = sweep.sweep(adc_sz, #(ro_dac_lengths, ro_pulse_generator_inst.set_ro_length, 'Readout pulse length'), 
                    (ro_dac_amplitudes, ro_pulse_generator_inst.set_ro_amplitude, 'Readout pulse amplitude'))
ro_dac_amplitude = ro_amplitude_calib['Calibrated fidelity'][1][0][np.argmax(ro_amplitude_calib['Calibrated fidelity'][2])]

adc.set_software_nums_multi(1)

Started at:  Jan 19 2018 17:32:54


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]
D:\qtlab_replacement\scripts\tomography.py:123: RuntimeWarning: invalid value encountered in true_divide
  probabilities = hists/hist_all



Elapsed time: 0 h 7 m 29.12 s


True

In [39]:
#setting optimal settings 
print ('Calibrated optimal readout amplitude: ', ro_dac_amplitude)
adc.set_software_nums_multi(5)
ro_pulse_generator_inst.set_ro_amplitude(ro_dac_amplitude)
adc_sz.measure()
adc.set_software_nums_multi(1)
plt.plot(adc_sz.calib_proba_points, adc_sz.calib_hists.T)
adc_calibrated_reducer = data_reduce.data_reduce(adc)
adc_calibrated_reducer.filters['sz'] = adc_sz.filter

Calibrated optimal readout amplitude:  1.0


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]
D:\qtlab_replacement\scripts\tomography.py:123: RuntimeWarning: invalid value encountered in true_divide
  probabilities = hists/hist_all


In [70]:
adc.stop()

### Rabi amplitude oscillations

In [66]:
ex_amps = np.linspace(0.03, 0.1,41)
#num_pulses = 50
# ex_amps = np.linspace(0,0.1,101)
num_pulses = 12
# ro_dac_length=0.2e-6
#readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
                [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

Started at:  Nov 24 2017 13:39:51


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]
root: ERROR    Spectrum_M3i2132 : Error 288 while setting reg 100 to 147468
root: ERROR    Spectrum_M3i2132 : Call: (SPC_M2CMD, M2CMD_CARD_START | M2CMD_CARD_ENABLETRIGGER | M2CMD_CARD_WAITREADY | M2CMD_DATA_WAITDMA) -> card is still running, access not possible                                                


SystemError: <built-in method write of _io.BufferedWriter object at 0x00000000157BA308> returned a result with an error set

In [ ]:
#ex_amps = np.linspace(0.3225,0.3325,21)
#num_pulses = 50
ex_amps = np.linspace(0.23,0.29,101)
num_pulses = 8
# ro_dac_length=0.2e-6
#readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
                [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

## Rabi amps fit

0.08333333333333333pi, amplitude: 0.12897510922465855
0.25pi, amplitude: 0.2647458629646688
0.4166666666666667pi, amplitude: 0.4016714104788954
0.5833333333333334pi, amplitude: 0.5410715160807286
0.75pi, amplitude: 0.6852557673186009
0.9166666666666666pi, amplitude: 0.8296049890956471

In [85]:
num_pulses = 12
ex_amps_first=np.linspace(0.03,0.6,81)
amps=set_amps(num_pulses,ex_amps_first)
angles_curve = fit_cal(amps)

ya tut first sweep


Started at:  Nov 24 2017 14:46:37


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 22 m 19.61 s
0.130656667026 0.0163320833782 [ 0.12699546  0.13062481  0.13425417  0.13788352  0.14151287  0.14514222
  0.14877157  0.15240093  0.15603028  0.15965963]


Started at:  Nov 24 2017 15:08:59


...


Elapsed time: 0 h 2 m 45.59 s
0.144184251238
angle: 0.75pi, amplitude: 0.6852557673186009


Started at:  Nov 24 2017 15:22:53


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 2 m 45.81 s
0.144349221777
angle: 0.9166666666666666pi, amplitude: 0.8296049890956471


IndexError: tuple index out of range

In [129]:
x=angles_curve[0]
y=angles_curve[1]
params_of_fit=angles_curve[2]
plt.plot(x,y,"-")
plt.plot([amps[0][0],amps[(len(amps)-1)][0]],[amps[0][1],amps[(len(amps)-1)][1]])
plt.plot([amps[i][0] for i in range(len(amps))],[amps[i][1] for i in range(len(amps))],'o')
plt.show()

[-0.0124978   0.05808691  0.08649733]
[1, 3, 5, 7, 9, 11] [0.12897510922465855, 0.26474586296466879, 0.40167141047889537, 0.54107151608072857, 0.68525576731860094, 0.82960498909564706]
[-0.0124978   0.05808691  0.08649733]
[1, 3, 5, 7, 9, 11] [0.12897510922465855, 0.26474586296466879, 0.40167141047889537, 0.54107151608072857, 0.68525576731860094, 0.82960498909564706]
[-0.0124978   0.05808691  0.08649733]
[1, 3, 5, 7, 9, 11] [0.12897510922465855, 0.26474586296466879, 0.40167141047889537, 0.54107151608072857, 0.68525576731860094, 0.82960498909564706]
[-0.012497800631844363, 0.05808691265648664, 0.086497330315593993]


In [103]:
from scipy.optimize import curve_fit

def set_ex_amp(amp,num_pulses):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
                [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
    
def func(x,a,b,c):
    return a*x**2+c*x+b

def func_for_fit_cal(x,a,b,c):
    return a*x**(5/3)+b*x**(4/3)+c

def fit_cal(data):
    x = [data[i][0] for i in range(len(data))]
    y = [data[i][1] for i in range(len(data))]
    popt = curve_fit(func_for_fit_cal,x,y)[0]
    angles = np.linspace(x[np.argmin(x)],x[np.argmax(x)],1000)
    amps = func_for_fit_cal(angles,popt[0],popt[1],popt[2])
    return angles,amps,[popt[0],popt[1],popt[2]]

def find_max(x,y):
    popt=curve_fit(func,x,y)[0]
    n1 = np.linspace(x[np.argmin(x)],x[np.argmax(x)],100)
    n2 = func(n1,popt[0],popt[1],popt[2])
    max_amp=n1[np.argmax(n2)]
    return max_amp

def first_sweep(num_pulses,ex_amps):# in order to find a period between first two peaks
    set_ex_first = lambda amp: set_ex_amp(amp,num_pulses)
    print('ya tut first sweep')
    measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_first, 'Rabi pulse amplitude'),
                                      filename=None,save=False)
    x = measurement['S21+'][1][0]
    y = (np.real(measurement['S21-'][2]),np.imag(measurement['S21-'][2]))
    fit= sin_fit(x,y)[0]
    x_max_0,x_min_0=(np.pi-fit[0])/(2*np.pi*fit[1]),(np.pi*4/2-fit[0])/(2*np.pi*fit[1])
    cut=np.abs((x_max_0-x_min_0))/4
    ex_amps=np.linspace(x_max_0-cut,x_max_0+cut,10)
    T0=1/fit[1]
    return T0,cut,ex_amps

def set_amps(num_pulses,ex_amps_first):
    T0,cut,ex_amps = first_sweep(num_pulses,ex_amps_first)
    print(T0,cut,ex_amps)
    amp_for_angle=[]
    k=0
    set_ex_next = lambda amp: set_ex_amp(amp,num_pulses)
    for i in range(int(num_pulses/2)):
        measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_next, 'Rabi pulse amplitude'),
                                      filename=None,save=False)
        x=measurement['S21-'][1][0]
        y=np.angle(measurement['S21-'][2])
        max_amp=find_max(x,y)
        amp_for_angle.append([(1+2*i),max_amp])
        if k>0:
            T=amp_for_angle[i][1]-amp_for_angle[i-1][1]
            print(T)
            ex_amps=np.linspace(max_amp+T-cut,max_amp+T+cut,10)
        else:
            ex_amps=np.linspace(max_amp+T0-cut,max_amp+T0+cut,10)
            k=1
        print('angle: {}pi, amplitude: {}'.format((1+2*i)/num_pulses,max_amp))
    return amp_for_angle

In [63]:
def sin_fit(x, y, resample=501):
	# фитует результаты измерений экспонентой
	def model(x, p):
		phase = p[0]
		freq = p[1]
		A = np.reshape(np.asarray(p[2:]),(len(p[2:]), 1))
		return A*np.cos(phase+x*freq*2*np.pi)
	
	cost = lambda p: (np.abs(model(x, p)-y)**2).ravel()
	
	means = np.reshape(np.mean(y, axis=1), (np.asarray(y).shape[0], 1))
	y = y-means
	
	ft = np.fft.fft(y-np.reshape(np.mean(y, axis=1), (y.shape[0], 1)), axis=1)/len(x)
	f = np.fft.fftfreq(len(x), x[1]-x[0])
	domega = (f[1]-f[0])*2*np.pi
	
	fR_id = np.argmax(np.sum(np.abs(ft)**2, axis=0))
	fR_id_conj = len(f)-fR_id
	if fR_id_conj > fR_id:
		tmp = fR_id_conj
		fR_id_conj = fR_id
		fR_id = tmp
	
	fR = np.abs((f[fR_id]))
	
	c = np.real(np.sum(ft[:,fR_id], axis=0))
	s = np.imag(np.sum(ft[:,fR_id], axis=0))
	phase = np.arctan2(s, c)
	#x0 = np.sqrt(np.mean(np.abs(ft[:,fR_id])**2)/np.mean(np.abs((ft[:,fR_id-1]+ft[:,fR_id+1])/2)**2)-1)/domega/2
	#print (np.abs(ft[:,fR_id])**2, np.abs((ft[:,fR_id-1]+ft[:,fR_id+1])/2)**2)
	
	A = np.sqrt(np.abs(ft[:,fR_id])**2+np.abs(ft[:,fR_id_conj])**2)
	p0 = [phase, fR]+A.tolist()
	
	from scipy.optimize import leastsq
	fitresults_all = []
	for random_id in range(100):
		random_samples = np.random.choice(len(x), int(len(x)*0.7))
		cost = lambda p: (np.abs(model(x[random_samples], p)-y[:,random_samples])**2).ravel()
		
		fitresults = leastsq (cost, p0)
		fitresults_all.append(fitresults[0])
	fitted_curve = model(x, fitresults[0])
	
	resampled_x = np.linspace(np.min(x), np.max(x), resample)
	resampled_y = model(resampled_x, fitresults[0])+means
	
	#for i in range(4):
	#	plt.figure(i)
	#	plt.plot(x, y[i,:], label='data')
	#	plt.plot(x, model(x, p0)[i,:], label='initial')
	#	plt.plot(x, model(x, fitresults[0])[i,:], label='fitted')
	#	plt.legend()
	
	return fitresults[0], fitted_curve+means, fitresults_all, resampled_x, resampled_y

## Rabi amplitude oscillations pi + pi/2 pulses

In [ ]:
from numpy import pi
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
#measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')
#measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
#def find_ampl(impulse):
    #i=0
    #derivative1, derivative2 =0,0
    #count=0
    #while (0.5*count<impulse/(0.5*pi)):
        #i=i+1;
        #derivative1=derivative2;
        #derivative2=(3*measurement['Mean Voltage (AC)'][i]-4*measurement[i-1]+measurement[i-2])/(2*(0.7/21)
        #if ((derivative1*derivative2)<0) :
            #count=count+1
    #return measurement[i]

#def Rabi_ampl(impulse):

# В таком сиде как оно написано здесь эта процедура не имеет никакого смысла =(
# Если утверждение amp*2=amp_pi верно, то это не нужно. Если нет, то таким обрзом вы это не увидите.

ex_amps = np.linspace(0,0.7,41)
num_pulses=4
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')
measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
print(fitted_parameters_rabi)
      # return find_ampl(impulse)
#ampl_half_pi=Rabi_ampl(pi/2)   
amp=(3*pi-fitted_parameters_rabi['phase'])/(2*pi*fitted_parameters_rabi['freq'])
amp_pi=(pi*5-fitted_parameters_rabi['phase'])/(2*pi*fitted_parameters_rabi['freq'])
amp, amp_pi

### Rabi Y-amplitude oscillations

In [ ]:
ex_amps = np.linspace(0.1,0.7, 101)
num_pulses = 5
#readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, 0, amp, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses +\
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi Y-pulse amplitude'), filename='Rabi Y-amplitude')

## Amplification of phase errors

In [130]:
amps

[[1, 0.12897510922465855],
 [3, 0.26474586296466879],
 [5, 0.40167141047889537],
 [7, 0.54107151608072857],
 [9, 0.68525576731860094],
 [11, 0.82960498909564706]]

In [132]:
num_pulses = np.arange(0, 25, 1)
y_sign = '+'
y_sign_mul = 1 if y_sign == '-' else -1

def set_ape_length(num_pulses):
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)]*num_pulses+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, y_sign_mul*amp, sigma, alpha), pg.p(None, pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (num_pulses, set_ape_length, 'APE identity pulse num'), filename='APE Y'+y_sign)

Started at:  Nov 24 2017 16:27:44


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 6 m 55.37 s


# Scanning over alpha and looking at phase error

In [18]:
adc.stop()

In [150]:
num_pulses = [4,8,12]
y_sign = '+'
y_sign_mul = 1 if y_sign == '+' else -1
#anharmonicity = -223e6
#alphas = np.linspace(-1.0, 1.0, 21)/(200e6*2*np.pi)
alphas = np.linspace(-2, -1, 21)
amp=amps[4][1]
def set_alpha(alpha_dimensionless,num_pulses):
    alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)]*num_pulses+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, y_sign_mul*amp, sigma, alpha), pg.p(None, pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

    pg.set_seq(sequence)
    awg_tek.run()
x=[]
y=[]
for i in num_pulses:    
    set_alphas = lambda amp: set_alpha(amp,i)
    measurement = sweep.sweep(adc_reducer, (alphas, set_alphas, 'HD DRAG alpha'),
                              filename='APE Y'+y_sign+' dependence on HD DRAG alpha',save=False)
    x.append(measurement['S21+'][1][0])
    y.append(np.angle(measurement['S21-'][2]))

Started at:  Nov 24 2017 17:30:23


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 5 m 49.13 s


Started at:  Nov 24 2017 17:36:14


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 5 m 47.62 s


Started at:  Nov 24 2017 17:42:03


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 5 m 47.3 s


In [151]:
x

[array([-2.  , -1.95, -1.9 , -1.85, -1.8 , -1.75, -1.7 , -1.65, -1.6 ,
        -1.55, -1.5 , -1.45, -1.4 , -1.35, -1.3 , -1.25, -1.2 , -1.15,
        -1.1 , -1.05, -1.  ]),
 array([-2.  , -1.95, -1.9 , -1.85, -1.8 , -1.75, -1.7 , -1.65, -1.6 ,
        -1.55, -1.5 , -1.45, -1.4 , -1.35, -1.3 , -1.25, -1.2 , -1.15,
        -1.1 , -1.05, -1.  ]),
 array([-2.  , -1.95, -1.9 , -1.85, -1.8 , -1.75, -1.7 , -1.65, -1.6 ,
        -1.55, -1.5 , -1.45, -1.4 , -1.35, -1.3 , -1.25, -1.2 , -1.15,
        -1.1 , -1.05, -1.  ])]

In [152]:
y

[array([-2.35670381, -2.35542892, -2.35320157, -2.35296134, -2.35434595,
        -2.35952332, -2.35950007, -2.352221  , -2.35456157, -2.35022207,
        -2.35403008, -2.35171017, -2.35447784, -2.35421385, -2.35527554,
        -2.35109509, -2.35508437, -2.35291221, -2.3530414 , -2.35114564,
        -2.35040061]),
 array([-2.35698858, -2.35425779, -2.35682433, -2.35527381, -2.35735203,
        -2.35674604, -2.35699724, -2.35935601, -2.3549887 , -2.3563816 ,
        -2.35912117, -2.35544876, -2.35743454, -2.35723302, -2.35686657,
        -2.35837166, -2.35832311, -2.35559143, -2.35929244, -2.35533608,
        -2.35966674]),
 array([-2.35650253, -2.3578985 , -2.35598664, -2.35715135, -2.35343623,
        -2.35549264, -2.35711806, -2.35702596, -2.35760488, -2.35637483,
        -2.35791028, -2.35558398, -2.35587833, -2.3547332 , -2.35279535,
        -2.35343958, -2.35344013, -2.35237856, -2.35811965, -2.36082736,
        -2.36012378])]

### X-Y axes angle error

In [ ]:
num_pulses = np.arange(0, 21, 1)
y_sign = '-'
y_sign_mul = 1 if y_sign == '-' else -1

def set_xyea_length(num_pulses):
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),]*(2*num_pulses)+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, y_sign_mul*amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (num_pulses, set_xyea_length, 'X-Y axes angle error pulse num'), filename='XY angle error witness'+y_sign)

### Optimizing X-Y axes angle 

In [ ]:
#angle_corrs = np.linspace(-np.pi/40, np.pi/40, 21)
angle_corrs = np.linspace(-0.03, 0.03, 21)
num_pulses = 20

for signs in ['++++', '+++-', '++-+', '++--', '+-++', '+-+-', '+--+', '+---', 
              '-+++', '-++-', '-+-+', '-+--', '--++', '--+-', '---+', '----']:
    sign_muls = [1 if s == '-' else -1 for s in signs]

    def set_xyea_angle_corr(angle_corr):
        sequence = [pg.p('iq_ex', length, pg.gauss_hd, sign_muls[0]*amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
                   [pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[1]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[1]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, sign_muls[2]*amp, 0, sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, sign_muls[2]*amp, 0, sigma, alpha), pg.p(None, pause_length),]*(2*num_pulses)+\
                   [pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[3]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('ro_trg', trg_length, pg.rect, 1), 
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

        pg.set_seq(sequence)
        awg_tek.run()
    measurement = sweep.sweep(adc_reducer, (angle_corrs, set_xyea_angle_corr, 'X-Y axes angle detuning (radians)'), filename='XY angle error optimization'+signs)

## Characterizing readout of |0>, |1>, |+>, |->, |i+>, |i-> states

In [40]:
state_prep_seqs = {'|0>': [],
                   '|+>': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|1>': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),\
                           pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|->': [pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|i+>': [pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y, sigma, alpha), pg.p(None, pause_length)],
                   '|i->': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_y,sigma, alpha), pg.p(None, pause_length)]}

plt.figure('State readout histograms gaussian')
sz_meas = {}
for state_name, state_prep_seq in state_prep_seqs.items():
    pg.set_seq(state_prep_seq+adc_sz.ro_seq)
    sz_meas[state_name] = np.real(adc_calibrated_reducer.measure()['sz'])
    hist, bin_edges = np.histogram(sz_meas[state_name], bins=adc_sz.calib_proba_points)
    proba_points = (bin_edges[1:]+bin_edges[:-1])/2
    plt.plot(proba_points, hist,label=state_name)
    print ('{0} state sz mean: {1}, std: {2}'.format(state_name, np.mean(sz_meas[state_name]), np.std(sz_meas[state_name])))
    
plt.legend()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


|0> state sz mean: -0.49075580016018117, std: 3.1846651908098624
|+> state sz mean: 0.02203556240676348, std: 3.1565782611953646
|1> state sz mean: 0.501142533487277, std: 3.0800372750869287
|-> state sz mean: 0.014423155258214759, std: 3.1712216652093104
|i+> state sz mean: -0.13336733346180996, std: 3.1901079173772757
|i-> state sz mean: -0.12533298024075062, std: 3.1862194994972866


In [ ]:
adc.stop()

### Tomography

In [41]:
#+ 
#amp=0.327
#amp2=0.657
observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Xo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_y, sigma, alpha), pg.p(None, pause_length)]+ro_seq, 
                 'operator': observables['X']},
            'Yo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['Y']},
            '-Xo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-X']},
            '-Yo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-Y']},
            'Zo': {'pulses':ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'x':{'operator':observables['X']},
                      'y':{'operator':observables['Y']},
                      'z':{'operator':observables['Z']}}
tomo = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [42]:
#+Gh
proj_seq = {'Z': {'pulses':ro_seq, 'operator':observables['Z']}}
reconstruction_basis={'z':{'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [43]:
## tomography of |0> state
#adc.set_software_nums_multi(20)
tomo.set_prepare_seq([])
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-Xo': array(-0.01614),
 '-Yo': array(0.00696),
 'Xo': array(-0.0144),
 'Yo': array(0.01106),
 'Zo': array(-0.05846),
 'x': 0.0008700000000000025,
 'y': 0.002049999999999998,
 'z': -0.058460000000000012}

# Clifford group

In [44]:
#+
import clifford
imp.reload(clifford)

pi2 = {'X/2': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]])},
       'Y/2': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]])},
       '-X/2':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)],
              'unitary': np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]])},
       '-Y/2':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_y, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])},
       'I':   {'pulses':[], 'unitary':np.asarray([[1, 0], [0,1]])},
       'X': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp2, 0, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.asarray([[0, 1],  [1, 0]])},
       'Y': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp2, sigma, alpha), pg.p(None, pause_length)],
               'unitary': 1j*np.asarray([[0, -1],  [1, 0]])}}
clifford_group = clifford.generate_group(pi2)
print(clifford_group.keys(), len(clifford_group))

dict_keys(['X/2', 'Y/2', '-X/2', '-Y/2', 'I', 'X', 'Y', 'Y/2 X/2', '-Y/2 X/2', 'Y X/2', 'X/2 Y/2', 'X/2 Y/2 X/2', '-X/2 Y/2', '-X/2 Y/2 X/2', 'X Y/2', '-Y/2 X/2 Y/2', 'Y/2 -X/2', 'Y/2 -X/2 Y/2', '-Y/2 -X/2', 'Y -X/2', 'X/2 -Y/2', '-X/2 -Y/2', 'X -Y/2', 'Y X']) 24


In [81]:
print([[ord(c) for c in k] for k in clifford_group.keys()])
print([ord (c) for c in 'X -Y/2'])

[[88, 47, 50], [89, 47, 50], [45, 88, 47, 50], [45, 89, 47, 50], [73], [45, 88], [45, 89], [89, 47, 50, 32, 88, 47, 50], [45, 89, 47, 50, 32, 88, 47, 50], [45, 89, 32, 88, 47, 50], [88, 47, 50, 32, 89, 47, 50], [88, 47, 50, 32, 89, 47, 50, 32, 88, 47, 50], [45, 88, 47, 50, 32, 89, 47, 50], [45, 88, 47, 50, 32, 89, 47, 50, 32, 88, 47, 50], [45, 88, 32, 89, 47, 50], [45, 89, 47, 50, 32, 88, 47, 50, 32, 89, 47, 50], [89, 47, 50, 32, 45, 88, 47, 50], [89, 47, 50, 32, 45, 88, 47, 50, 32, 89, 47, 50], [45, 89, 47, 50, 32, 45, 88, 47, 50], [45, 89, 32, 45, 88, 47, 50], [88, 47, 50, 32, 45, 89, 47, 50], [45, 88, 47, 50, 32, 45, 89, 47, 50], [45, 88, 32, 45, 89, 47, 50], [45, 89, 32, 45, 88]]
[88, 32, 45, 89, 47, 50]


# Randomized benchmarking

In [45]:
#+
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)
imp.reload(sweep)

adc.set_software_nums_multi(4)
bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

#unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
#             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
#             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
#             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}
bench.interleavers = clifford_group
#bench.sequence_length = 10
bench.random_sequence_num = 20
#bench.prepare_random_interleaving_sequences()
#bench.get_points()
#bench.generate_interleaver_sequence_from_names()

In [46]:
adc.stop()

In [47]:
#+
#ro_adc_length=0.2e-6 #ВНИМАНИЕ! я это удалю (закомментирую) тут, т.к. оно всё равно здесь ничего не делает, только мозги выносит
seq_lengths = np.arange(0,21, 1)
#seq_lengths = np.arange(0, 100, 10)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
clifford_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

Started at:  Nov 24 2017 00:29:07


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 1 h 56 m 37.06 s


In [ ]:
#+
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 
         1-clifford_bench['distance'][2], 
         marker='o', 
         color='black', 
         linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1))
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))

plt.plot(clifford_bench['distance'][1][0],
         clifford_expfit[0],
         label='Clifford set')
plt.grid()
plt.xlabel('Number of gates')
plt.ylabel('Fidelity')
plt.legend()
print('Clifford fidelity: {0:6.3f}'.format(np.exp(-1/clifford_fitresults[0])))

In [ ]:
clifford_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\21-07-16'
name='distance None'
clifford_bench = {}
clifford_bench = save_pkl.load_pkl(name, clifford_location)[1]
#clifford_bench['distance']
clifford_bench['Pulse sequence']

In [ ]:
print([clifford_bench['Pulse sequence'][2][1][i][::2] for i in range(len(clifford_bench['Pulse sequence'][2][1]))])
print([clifford_bench['Pulse sequence'][2][1][i][::2] for i in range(len(clifford_bench['Pulse sequence'][2][1]))])

In [ ]:
clifford_bench['distance'][2][1,:]

In [ ]:
not_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\22-14-16'
name='distance None'
not_bench = {}
not_bench = save_pkl.load_pkl(name, not_location)[1]
#clifford_bench['distance']
pass

In [ ]:
hadamard_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\21-40-30'
name='distance None'
hadamard_bench = {}
hadamard_bench = save_pkl.load_pkl(name, hadamard_location)[1]
#clifford_bench['distance']
pass

## interleaved benchmarking of H gate and NOT

In [48]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

bench.interleavers = clifford_group
bench.random_sequence_num = 20
bench.target_gate = clifford_group['X -Y/2']['pulses'] # NOT
bench.target_gate_name = 'Hadamard'
bench.target_gate_unitary = clifford_group['X -Y/2']['unitary']


In [49]:
seq_lengths = np.arange(0, 21, 1)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
hadamard_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

Started at:  Nov 24 2017 02:25:56


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 1 h 56 m 59.14 s


In [ ]:
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 1-clifford_bench['distance'][2], marker='o', color='black', linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1), label='Cliffords (average)')
plt.plot(hadamard_bench['distance'][1][0], 1-hadamard_bench['distance'][2], marker='o', color='blue', linestyle='')
#plt.plot(hadamard_bench['distance'][1][0], np.mean(1-hadamard_bench['distance'][2], axis=1), label='Hadamard interleaved (average)')
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))
hadamard_fitresults, hadamard_expfit = fitting.exp_fit1d(hadamard_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-hadamard_bench['distance'][2], axis=1), 
                                                                   (1, len(hadamard_bench['distance'][1][0]))))
plt.plot(clifford_bench['distance'][1][0],clifford_expfit[0], label='Cliffords (fit)')
plt.plot(hadamard_bench['distance'][1][0],hadamard_expfit[0], label='Hadamard (fit)')#, linestyle='-', marker='')
print('''Clifford fidelity: {0:6.3f}
Hadamard+Clifford fidelity: {1:6.3f}
Hadamard error rate: {2:6.3f}'''.format(np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/hadamard_fitresults[0]),
                                       1-np.exp(-1/hadamard_fitresults[0])/np.exp(-1/clifford_fitresults[0])))
plt.xlabel(clifford_bench['distance'][0][0])
plt.ylabel('Fidelity')
plt.legend()
plt.grid()

In [50]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

bench.interleavers = clifford_group
bench.random_sequence_num = 20
bench.target_gate = clifford_group['X']['pulses'] # NOT
bench.target_gate_name = 'NOT'
bench.target_gate_unitary = clifford_group['X']['unitary']


In [51]:
seq_lengths = np.arange(0, 21, 1)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
not_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

Started at:  Nov 24 2017 04:23:06


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 1 h 57 m 28.4 s


In [53]:
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 1-clifford_bench['distance'][2], marker='o', color='black', linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1), label='Cliffords (mean)')
plt.plot(hadamard_bench['distance'][1][0], 1-hadamard_bench['distance'][2], marker='o', color='blue', linestyle='')
#plt.plot(hadamard_bench['distance'][1][0], np.mean(1-hadamard_bench['distance'][2], axis=1), label='H interleaved (mean)')
plt.plot(not_bench['distance'][1][0], 1-not_bench['distance'][2], marker='o', color='red', linestyle='')
#plt.plot(not_bench['distance'][1][0], np.mean(1-not_bench['distance'][2], axis=1), label='NOT interleaved (mean)')
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))
hadamard_fitresults, hadamard_expfit = fitting.exp_fit1d(hadamard_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-hadamard_bench['distance'][2], axis=1), 
                                                                   (1, len(hadamard_bench['distance'][1][0]))))
not_fitresults, not_expfit = fitting.exp_fit1d(not_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-not_bench['distance'][2], axis=1), 
                                                                   (1, len(not_bench['distance'][1][0]))))
plt.plot(clifford_bench['distance'][1][0],clifford_expfit[0], label='Cliffords (fit)')
plt.plot(hadamard_bench['distance'][1][0],hadamard_expfit[0], label='Hadamard (fit)')#, linestyle='-', marker='')
plt.plot(not_bench['distance'][1][0],not_expfit[0], label='NOT (fit)')
print('''Clifford fidelity: {0:6.3f}
Hadamard+Clifford fidelity: {1:6.3f}
NOT+Clifford fidelity: {3:6.3f}
Hadamard error rate: {2:6.3f}
NOT error rate: {4:6.3f}'''.format(np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/hadamard_fitresults[0]),
                                       1-np.exp(-1/hadamard_fitresults[0])/np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/not_fitresults[0]),
                                       1-np.exp(-1/not_fitresults[0])/np.exp(-1/clifford_fitresults[0])))
plt.xlabel(clifford_bench['distance'][0][0])
plt.ylabel('Fidelity')
plt.legend()
plt.grid()

Clifford fidelity:  0.934
Hadamard+Clifford fidelity:  0.838
NOT+Clifford fidelity:  0.882
Hadamard error rate:  0.102
NOT error rate:  0.055


In [ ]:
bench.target_gate = bench.interleavers['X']['pulses']
bench.target_gate_unitary = bench.interleavers['X']['unitary']
bench.target_gate_name = 'X (benchmarking)'
tfunc = lambda : np.mean(bench.measure()['Euclidean distance'])
class hd_pulse_generator:
    def __init__(self, pulse_setter, pg, channel):
        self.amp_x = -amp_x_hd
        self.amp_y = amp_x_hd
        self.sigma = sigma_hd
        self.length = length_hd
        self.pause_length = pause_length_hd
        self.alpha = alpha_hd
        
        self.pg = pg
        self.channel = channel
        self.pulse = self.pulse_gen()
        self.pulse_setter = pulse_setter
    def set_real_amplitude(self,x):
        self.amp_x = x
        self.pulse_setter(self.pulse_gen())
    def set_imag_amplitude(self,y):
        self.amp_y = y
        self.pulse_setter(self.pulse_gen())
    def set_sigma(self,s):
        self.sigma = s
        self.pulse_setter(self.pulse_gen())
    def set_length(self,l):
        self.length = l
        self.pulse_setter(self.pulse_gen())
    def set_pause_length(self, p):
        self.pause_length = p
        self.pulse_setter(self.pulse_gen())
    def set_alpha(self, a):
        self.alpha = a
        self.pulse_setter(self.pulse_gen())
    def pulse_gen(self):
        return [pg.p(self.channel, self.length, self.pg.gauss_hd, self.amp_x+1j*self.amp_y, 0, self.sigma, self.alpha), pg.p(None, self.pause_length)]
    
xg = hd_pulse_generator(bench.set_target_pulse, pg, 'iq_ex')

In [ ]:
sol, score = sweep.optimize(tfunc,  (xg.set_real_amplitude, xg.amp_x), 
                                    (xg.set_imag_amplitude, xg.amp_y),
                                    (xg.set_alpha, xg.alpha))

In [ ]:
amps = np.linspace(0.925, 0.975, 21)
alphas = np.linspace(-2e-9, 2e-9, 11)
sweep.sweep(bench, (amps, xg.set_real_amplitude, 'Amplitude'), (alphas, xg.set_alpha, 'HD DRAG coeff'), filename='Randomized interleaved benchmarging X-rotation')

In [ ]:
data = save_pkl.load_pkl('Euclidean distance Randomized interleaved benchmarging X-rotation', 'D:\\qtlab\\ReiData\\data\\2017-07-02\\23-03-34')
plt.figure()
plt.pcolormesh(data[1]['Euclidean distance'][1][1], data[1]['Euclidean distance'][1][0], data[1]['Euclidean distance'][2][:,:,5])
plt.colorbar()

In [ ]:
data[1]

In [ ]:
data[1]['Euclidean distance']

In [ ]:
meas = bench.measure()

In [ ]:
meas['Xo']

In [ ]:
import plotting
imp.reload(plotting)
plotting.plot_measurement(bench.reference_benchmark_result)

# Optimizing Drag

In [ ]:
num_interleved_pulses = 2
alphas = np.linspace(-1e-8, 1e-8, 201)
def set_drag (alpha):
    ramsey_half = [pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                 pg.p(None, pause_length_hd)]
    interleaved_identity = [pg.p('iq_ex', length_hd, pg.gauss_hd, -amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd),
                           pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd)]
    tomo.set_prepare_seq(ramsey_half+interleaved_identity*num_interleved_pulses+ramsey_half)
    
measurement = sweep.sweep(tomo, (alphas, set_drag, 'delta_freq'), filename='DRAG', output=False)

### Randomized Benchmarking

In [ ]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.linspace(0, 100, 11)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [ ]:
rho_m

In [ ]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [ ]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
m

In [ ]:
fidelities

In [ ]:
psis

In [ ]:
rho_m

In [ ]:
rho_t

In [ ]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

In [ ]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

In [ ]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

In [ ]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

In [ ]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

### Optimizing phase between x and y pulses

In [ ]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

In [ ]:
phases[7]

In [ ]:
plt.plot(np.asarray(results).T)

In [ ]:
adc.stop()

In [ ]:
plt.close()

# load calibration data

In [ ]:
import pickle
f0 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-46\\Voltage ro calibration.pkl', 'rb')
f1 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-14\\Voltage ro calibration.pkl', 'rb')
cal0 = pickle.load(f0)
cal1 = pickle.load(f1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt4
cal0_nomean = cal0[1]['Voltage'][2][0,:,:]-np.mean(cal0[1]['Voltage'][2][0,:,:])
cal1_nomean = cal0[1]['Voltage'][2][1,:,:]-np.mean(cal0[1]['Voltage'][2][1,:,:])

cal0_mean = np.mean(cal0_nomean, axis=0)
cal1_mean = np.mean(cal1_nomean, axis=0)

In [ ]:
cal_mean = (cal0_mean+cal1_mean)/2.
cal_diff = (cal1_mean-cal0_mean)

plt.figure('Samples')
plt.plot(np.mean(cal0_nomean, axis=0), label='|0>')
plt.plot(np.mean(cal1_nomean, axis=0), label='|1>')
plt.legend()

plt.figure('Offset & feature')
plt.plot(cal_mean, label='mean')
plt.plot(cal_diff, label='diff')
plt.legend()

In [ ]:
plt.figure('Fourier domain')
plt.plot(np.abs(np.fft.fft(cal0_mean)), label='|0>')
plt.plot(np.abs(np.fft.fft(cal1_mean)), label='|1>') 
plt.plot(np.abs(np.fft.fft(cal_mean)), label='mean')
plt.plot(np.abs(np.fft.fft(cal_diff)), label='diff')
plt.legend()

feature = np.conj(cal_diff/np.sum(np.abs(cal_diff**2)))

In [ ]:
coeffs0 = np.dot(cal0_nomean-cal_mean, feature)
coeffs1 = np.dot(cal1_nomean-cal_mean, feature)
predictions = np.asarray([np.real(coeffs0), np.real(coeffs1)])

In [ ]:
hist_all, bins = np.histogram(predictions, bins='auto')
proba_points = (bins[1:]+bins[:-1])/2.
hists = []
for y in range(2):
    hists.append(np.histogram(predictions[y,:], bins=bins)[0])

hists = np.asarray(hists, dtype=float)
probabilities = hists/hist_all
naive_probabilities = np.asarray([proba_points<0, proba_points>0], dtype=float)
probabilities[np.isnan(probabilities)] = naive_probabilities[np.isnan(probabilities)]
predictor = lambda x: np.interp(x, proba_points, probabilities[1,:], left=0., right=1.)
calib_proba_points = proba_points
calib_proba = probabilities[1,:]
calib_hists = hists

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
roc = roc_curve([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))
roc_auc = roc_auc_score([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))

In [ ]:
plt.figure('ROC')
plt.plot(roc[0], roc[1])
plt.xlabel('True Positive Rate')
plt.ylabel('True Negative Rate')
roc_auc

In [ ]:
plt.figure('probability curve')
plt.plot(calib_proba_points, calib_proba)

In [ ]:
plt.figure('Readout hists')
plt.bar(proba_points-0.25, hists[0], width=0.05, label='|0>')
plt.bar(proba_points+0.25, hists[1], width=0.05, label='|1>')
plt.legend()

In [ ]:
plt.close()

In [ ]:
cal0_nomean.shape

In [ ]:
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal0_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal1_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')

In [ ]:
plt.plot(np.mean(cal0_nomean, axis=0))
plt.plot(np.mean(cal1_nomean, axis=0))